In [1]:
# Directorio en Databricks File System con los archivos a procesar
directorio = "/FileStore/tables/5qafb5rr1493121316812/"

In [2]:
archivos = dbutils.fs.ls(directorio)
nombres=[]
for archivo in archivos:
  nombres.append(archivo.name)
print nombres

In [3]:
# -*- coding: utf-8 -*-

from string import punctuation
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from datetime import *
import pandas as pd
from pyspark import SQLContext

sqlc = SQLContext(sc)

In [4]:
dataFrame_dicci = sqlContext.sql("SELECT * FROM diccionario")
diccionario = dataFrame_dicci.select("palabra").rdd.flatMap(lambda x: x).collect()

dataFrame_stop = sqlContext.sql("SELECT * FROM stop")
sp_stopwords = dataFrame_stop.select("stpword").rdd.flatMap(lambda x: x).collect()

#Código unicode correspondiente a los caracteres válidos para filtrar el texto.
puntuacion_si = [32, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 193, 201, 205, 199, 209, 211, 218, 220, 225, 231, 233, 237, 241, 243, 250, 252]

In [5]:
###############
#######Fuciones
###############

# Filtrado de palabras compuestas o simples que empiecen por mayúscula. 
def buscarPalabrasCompuestas(texto):
  mayusculas = [ 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Ñ', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
  anterior = False
  palabraaux = ""
  listabpc = texto.split(".")
  compuestas = []
  for i in listabpc:
      listabpc2 =i.split(',')

      for j in listabpc2:
          listabpc3 = j.split(" ")

          for ind,k in enumerate(listabpc3):
              
              if k != "":
              
                  if k[0] in mayusculas:
                      if anterior == False:

                          palabraaux += k
                      else:
                          palabraaux += " "
                          palabraaux += k

                      anterior = True
                      if ind == (len(listabpc3)-1):
                          compuestas.append(palabraaux)
                          palabraaux = ""
                          anterior = False

                  if k[0] not in mayusculas:
                      if anterior == True  :
                          compuestas.append(palabraaux)
                          palabraaux = ""
                      anterior = False


  return compuestas

# Creacion de una lista ordenada con el número de veces que aparece la palabra del diccionario en el texto.
def palabrasCompuestas(texto):
  lista = []
  for i in diccionario:
    a = texto.count(i)
    lista.append(a)   
  return lista

# Suma de los valores que genera el metodo palabrasCompuestas() para cada una de las palabras del diccionario.
def Contar(acum,n):
  for j in range(len(diccionario)):
    acum[j] += n[j]
  return acum

# Limpieza del texto de palabras dentro de diccionario.
def eliminarPalabrasComp(texto):
  for palabra in diccionario:
    texto = texto.replace(palabra, "")
  return texto


In [6]:
##################
#####Procedimiento
##################

for fichero in nombres:
  
  mifich = sc.textFile(directorio + fichero)
  
  previo = mifich.flatMap(buscarPalabrasCompuestas)
  
  previo1 = previo.map(lambda palabra : ''.join([char for char in palabra if ord(char) in puntuacion_si]))
  
  previo2 = previo1.map(lambda x : (x,1)).reduceByKey(lambda x,y : x+y).collect()
  
  #Creamos un dataframe de pandas vacio que se utlilizara para almacenar temporalmente las palabras de la lista diccionario.
  df = pd.DataFrame(columns = ['palabra'])
  #Casting de candidatas a palabras compuestas para introducir en el diccionario.
  for palabra in previo2:
    if palabra[1] >= 4 and palabra[0] not in diccionario and palabra[0].lower().encode('utf8') not in list(map(lambda x: x.encode('utf8'), sp_stopwords)):
      diccionario.append(palabra[0])     
      data = pd.DataFrame({'palabra':[palabra[0]]})
      df = df.append(data)
  
  if not df.empty:
    sqlc.createDataFrame(df).write.insertInto("diccionario")
  
  lista2 = mifich.map(palabrasCompuestas)

  lista3 = lista2.reduce(Contar) 
  
  tran1 = mifich.map(eliminarPalabrasComp)
  
  tran2 = tran1.map(lambda reg : ''.join([char for char in reg if ord(char) in puntuacion_si])) 

  tran3 = tran2.flatMap(lambda reg : reg.split(' ')).map(lambda x : x.lower())

  tran4 = tran3.filter(lambda palabra : palabra not in sp_stopwords)

  tran5 = tran4.map(lambda x : (x,1))

  salida = tran5.reduceByKey(lambda x,y : x+y).toDF(['clave','valor'])
  
  datos = fichero.replace(".txt","").split("_")
  fecha = date(int(datos[1]), int(datos[2]), int(datos[3]))
  fecha = fecha.isocalendar()

  categoria = datos[0]
  anio = datos[1]
  mes = datos[2]
  dia = datos[3]
  semana = fecha[1]
  dia_semana = fecha[2]
  

  lista_comp = []
  for i in range(len(diccionario)):
    if lista3[i] != 0:
      lista_comp.append((diccionario[i],lista3[i]))
  
  rdd_comp = sc.parallelize(lista_comp)
  salida_comp = rdd_comp.toDF(['clave','valor'])

  salida_comp = salida_comp.withColumn("categoria", lit(categoria)).withColumn("anio", lit(anio)).withColumn("mes", lit(mes)).withColumn("dia", lit(dia)).withColumn("semana", lit(semana)).withColumn("dia_semana", lit(dia_semana))
  
  salida = salida.withColumn("categoria", lit(categoria)).withColumn("anio", lit(anio)).withColumn("mes", lit(mes)).withColumn("dia", lit(dia)).withColumn("semana", lit(semana)).withColumn("dia_semana", lit(dia_semana))

  
  salida.write.insertInto("contadorDB.palabrastexto")
  salida_comp.write.insertInto("contadorDB.palabrastexto")
  

In [7]:
###############
##Mapa de Calor
###############

def mapaPalabras(texto):
  principal = False
  c=0

  posicion1 = 0
  posicion2 = 0
  valor = 0.0
  salida = []
  texto = texto.split(" ")
  for auxiliar in palabra2:

      c=0
      posicion1 = 0
      posicion2 = 0
      valor = 0.0
      principal = False

      for ind,i in enumerate(texto):

          i = i.replace(",","").replace(".","")


          if i == palabra.decode('utf8'):

              posicion1= ind
              if principal == False and posicion2 !=0:
                  c+=1
                  valor = valor+(posicion1-posicion2)
              principal = True


          if i == auxiliar.decode('utf8') :

              posicion2 = ind
              if principal == True:
                  c+=1
                  valor = valor+(posicion2-posicion1)
              principal = False

      if c !=0:
          valor = valor/c

          salida.append((palabra, auxiliar, valor,c))

  return salida

# palabra principal respecto a la que hacer el mapa de calor
palabra = "Madrid"

# lista de palabras con las que se quiere comparar la principal
palabra2= ["Granada", "Ponferrada", "Valencia", "Barcelona", "Sevilla","Messi", "partido"]

mifich = sc.textFile(directorio + nombres[0])
salidaMapa2 = mifich.flatMap(mapaPalabras)
salidaMapa3 = salidaMapa2.collect()

sal = salidaMapa3 

listaMapaAux = []
minimo = 100000
maximo = 0

print sal

for i in palabra2:
    valor=0
    cantidad = 0

    for lista in sal:
        if lista !=[]:
            
          if lista[1] == i:
            valor += lista[2]*lista[3]
            cantidad += lista[3]
                        
    if cantidad !=0:
        listaMapaAux.append((i,valor/cantidad))

for i in listaMapaAux:
    if i[1] < minimo:
        minimo = i[1]
    if i[1] > maximo:
        maximo = i[1]
        
listaMapaAux2 = []
for i in listaMapaAux:
    
    if (maximo-minimo) != 0:
        listaMapaAux2.append((palabra, i[0], str(int(round(i[1]))),str(int(round(100-((i[1]-minimo)/(maximo-minimo))*100)))))
    else:
        listaMapaAux2.append((palabra ,i[0], str(int(round(i[1]))),str(int(round(100-((i[1]-minimo)/(1))*100)))))
        
listaMapaAux2 = sc.parallelize(listaMapaAux2).toDF(["Clave","Palabra","Distancia","Relación"])
listaMapaAux2.show()